### Carga de la tabla results_movie en la capa Gold

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS movie_gold.results_movies (
    year_release_date INT,
    country_name STRING,
    company_name STRING,
    budget DOUBLE,
    revenue DOUBLE,
    movie_id INT,
    country_id  INT,
    company_id INT,
    created_date DATE,
    updated_date DATE)
    USING DELTA
""")

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW v_results_movie
    AS
    SELECT M.year_release_date, C.country_name, PCO.company_name,
        M.budget, M.revenue, M.movie_id, C.country_id, PCO.company_id
    FROM movie_silver.movies M
    INNER JOIN movie_silver.productions_countries PC ON M.movie_id = PC.movie_id
    INNER JOIN movie_silver.countries C ON PC.country_id = C.country_id
    INNER JOIN movie_silver.movies_companies MC ON M.movie_id = MC.movie_id
    INNER JOIN movie_silver.productions_companies PCO ON MC.company_id = PCO.company_id
    WHERE M.file_date = '{v_file_date}'
""")

In [0]:
spark.sql("""
    MERGE INTO movie_gold.results_movies AS target
    USING v_results_movie AS source
        ON (    target.movie_id = source.movie_id
            AND target.country_id = source.country_id
            AND target.company_id = source.company_id)
    WHEN MATCHED THEN
    UPDATE SET
        target.year_release_date = source.year_release_date,
        target.country_name = source.country_name,
        target.company_name = source.company_name,
        target.budget = source.budget,
        target.revenue = source.revenue,
        target.movie_id = source.movie_id,
        target.country_id = source.country_id,
        target.company_id = source.company_id,
        target.updated_date = CURRENT_DATE()
    WHEN NOT MATCHED THEN
    INSERT (year_release_date, country_name, company_name, budget, revenue, movie_id, country_id, company_id, created_date)
    VALUES (source.year_release_date, 
            source.country_name, 
            source.company_name, 
            source.budget, 
            source.revenue,
            source.movie_id,
            source.country_id,
            source.company_id, 
            CURRENT_DATE()
            )
    """)

In [0]:
%sql
--SELECT COUNT(*) FROM movie_gold.results_movies
--drop table  movie_gold.results_movies 
SELECT created_date, updated_date, count(*)
FROM movie_gold.results_movies
GROUP BY created_date, updated_date